<a href="https://colab.research.google.com/github/anirban1221/Learning_PyTorch/blob/main/next_word_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install nltk

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from collections import Counter
from torch.utils.data import Dataset, DataLoader
from nltk.tokenize import word_tokenize
import nltk

In [3]:
document = """About the Program
What is the course fee for  Data Science Mentorship Program (DSMP 2023)
The course follows a monthly subscription model where you have to make monthly payments of Rs 799/month.
What is the total duration of the course?
The total duration of the course is 7 months. So the total course fee becomes 799*7 = Rs 5600(approx.)
What is the syllabus of the mentorship program?
We will be covering the following modules:
Python Fundamentals
Python libraries for Data Science
Data Analysis
SQL for Data Science
Maths for Machine Learning
ML Algorithms
Practical ML
MLOPs
Case studies
You can check the detailed syllabus here - https://learnwith.campusx.in/courses/CampusX-Data-Science-Mentorship-Program-637339afe4b0615a1bbed390
Will Deep Learning and NLP be a part of this program?
No, NLP and Deep Learning both are not a part of this program’s curriculum.
What if I miss a live session? Will I get a recording of the session?
Yes all our sessions are recorded, so even if you miss a session you can go back and watch the recording.
Where can I find the class schedule?
Checkout this google sheet to see month by month time table of the course - https://docs.google.com/spreadsheets/d/16OoTax_A6ORAeCg4emgexhqqPv3noQPYKU7RJ6ArOzk/edit?usp=sharing.
What is the time duration of all the live sessions?
Roughly, all the sessions last 2 hours.
What is the language spoken by the instructor during the sessions?
Hinglish
How will I be informed about the upcoming class?
You will get a mail from our side before every paid session once you become a paid user.
Can I do this course if I am from a non-tech background?
Yes, absolutely.
I am late, can I join the program in the middle?
Absolutely, you can join the program anytime.
If I join/pay in the middle, will I be able to see all the past lectures?
Yes, once you make the payment you will be able to see all the past content in your dashboard.
Where do I have to submit the task?
You don’t have to submit the task. We will provide you with the solutions, you have to self evaluate the task yourself.
Will we do case studies in the program?
Yes.
Where can we contact you?
You can mail us at nitish.campusx@gmail.com
Payment/Registration related questions
Where do we have to make our payments? Your YouTube channel or website?
You have to make all your monthly payments on our website. Here is the link for our website - https://learnwith.campusx.in/
Can we pay the entire amount of Rs 5600 all at once?
Unfortunately no, the program follows a monthly subscription model.
What is the validity of monthly subscription? Suppose if I pay on 15th Jan, then do I have to pay again on 1st Feb or 15th Feb
15th Feb. The validity period is 30 days from the day you make the payment. So essentially you can join anytime you don’t have to wait for a month to end.
What if I don’t like the course after making the payment. What is the refund policy?
You get a 7 days refund period from the day you have made the payment.
I am living outside India and I am not able to make the payment on the website, what should I do?
You have to contact us by sending a mail at nitish.campusx@gmail.com
Post registration queries
Till when can I view the paid videos on the website?
This one is tricky, so read carefully. You can watch the videos till your subscription is valid. Suppose you have purchased subscription on 21st Jan, you will be able to watch all the past paid sessions in the period of 21st Jan to 20th Feb. But after 21st Feb you will have to purchase the subscription again.
But once the course is over and you have paid us Rs 5600(or 7 installments of Rs 799) you will be able to watch the paid sessions till Aug 2024.
Why lifetime validity is not provided?
Because of the low course fee.
Where can I reach out in case of a doubt after the session?
You will have to fill a google form provided in your dashboard and our team will contact you for a 1 on 1 doubt clearance session
If I join the program late, can I still ask past week doubts?
Yes, just select past week doubt in the doubt clearance google form.
I am living outside India and I am not able to make the payment on the website, what should I do?
You have to contact us by sending a mail at nitish.campusx@gmai.com
Certificate and Placement Assistance related queries
What is the criteria to get the certificate?
There are 2 criterias:
You have to pay the entire fee of Rs 5600
You have to attempt all the course assessments.
I am joining late. How can I pay payment of the earlier months?
You will get a link to pay fee of earlier months in your dashboard once you pay for the current month.
I have read that Placement assistance is a part of this program. What comes under Placement assistance?
This is to clarify that Placement assistance does not mean Placement guarantee. So we dont guarantee you any jobs or for that matter even interview calls. So if you are planning to join this course just for placements, I am afraid you will be disappointed. Here is what comes under placement assistance
Portfolio Building sessions
Soft skill sessions
Sessions with industry mentors
Discussion on Job hunting strategies
"""

In [4]:
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [5]:
tokens=word_tokenize(document.lower())

In [6]:
vocab={"<unk>":0}
for token in Counter(tokens).keys():
  if token not in vocab:
    vocab[token]=len(vocab)

vocab

{'<unk>': 0,
 'about': 1,
 'the': 2,
 'program': 3,
 'what': 4,
 'is': 5,
 'course': 6,
 'fee': 7,
 'for': 8,
 'data': 9,
 'science': 10,
 'mentorship': 11,
 '(': 12,
 'dsmp': 13,
 '2023': 14,
 ')': 15,
 'follows': 16,
 'a': 17,
 'monthly': 18,
 'subscription': 19,
 'model': 20,
 'where': 21,
 'you': 22,
 'have': 23,
 'to': 24,
 'make': 25,
 'payments': 26,
 'of': 27,
 'rs': 28,
 '799/month': 29,
 '.': 30,
 'total': 31,
 'duration': 32,
 '?': 33,
 '7': 34,
 'months': 35,
 'so': 36,
 'becomes': 37,
 '799': 38,
 '*': 39,
 '=': 40,
 '5600': 41,
 'approx': 42,
 'syllabus': 43,
 'we': 44,
 'will': 45,
 'be': 46,
 'covering': 47,
 'following': 48,
 'modules': 49,
 ':': 50,
 'python': 51,
 'fundamentals': 52,
 'libraries': 53,
 'analysis': 54,
 'sql': 55,
 'maths': 56,
 'machine': 57,
 'learning': 58,
 'ml': 59,
 'algorithms': 60,
 'practical': 61,
 'mlops': 62,
 'case': 63,
 'studies': 64,
 'can': 65,
 'check': 66,
 'detailed': 67,
 'here': 68,
 '-': 69,
 'https': 70,
 '//learnwith.campusx.i

In [7]:
len(vocab)

289

In [8]:
## finding out the sentences
input_sentences=document.split('\n')

In [9]:
def text_to_indices(sentence,vocab):
  numerical_sentences=[]
  for token in sentence:
    if token in vocab:
      numerical_sentences.append(vocab[token])

    else:
      numerical_sentences.append(vocab["<unk>"])

  return numerical_sentences

In [10]:
input_numerical_sentences=[]

for sentence in input_sentences:
  input_numerical_sentences.append(text_to_indices(word_tokenize(sentence.lower()),vocab))

In [11]:
len(input_numerical_sentences)

78

In [12]:
training_sequence=[]

for sentence in input_numerical_sentences:
  for i in range(1,len(sentence)):
    training_sequence.append(sentence[:i+1])

In [13]:
len(training_sequence)

942

In [14]:
training_sequence[:5]
## as we are seeing here the number of sentences in the training_sequence are different
## hence we have to make the sentences length same hence we have to pad the sequences



[[1, 2], [1, 2, 3], [4, 5], [4, 5, 2], [4, 5, 2, 6]]

In [15]:
len_list=[]
for sequence in training_sequence:
  len_list.append(len(sequence))

max_len=max(len_list)

In [16]:
padded_training_sequence=[]
for sequence in training_sequence:
  padded_training_sequence.append([0]*(max_len - len(sequence)) + sequence)

In [17]:
len(padded_training_sequence[10])

62

In [18]:
padded_training_sequence=torch.tensor(padded_training_sequence,dtype=torch.long)

In [19]:
padded_training_sequence

tensor([[  0,   0,   0,  ...,   0,   1,   2],
        [  0,   0,   0,  ...,   1,   2,   3],
        [  0,   0,   0,  ...,   0,   4,   5],
        ...,
        [  0,   0,   0,  ..., 285, 176, 286],
        [  0,   0,   0,  ..., 176, 286, 287],
        [  0,   0,   0,  ..., 286, 287, 288]])

In [20]:
X=padded_training_sequence[:,:-1]
y=padded_training_sequence[:,-1]

In [21]:
class Next_word_Dataset(Dataset):
  def __init__(self,X,y):
    self.X=X
    self.y=y

  def __len__(self):
    return self.X.shape[0]

  def __getitem__(self,index):
    return self.X[index], self.y[index]

In [22]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
train_dataloader=DataLoader(Next_word_Dataset(X_train,y_train),batch_size=32, shuffle=True)
test_dataloader=DataLoader(Next_word_Dataset(X_test,y_test),batch_size=32, shuffle=False)

In [23]:
class LstmModel(nn.Module):

  def __init__(self,vocab_size):
    super().__init__()
    self.embedding=nn.Embedding(vocab_size,100)
    self.lstm=nn.LSTM(100,150,batch_first=True)
    self.fc=nn.Linear(150,vocab_size)

  def forward(self,x):
    embedded=self.embedding(x)
    intermediate_hidden_states, (final_hidden_state, final_cell_state) = self.lstm(embedded)
    output = self.fc(final_hidden_state.squeeze(0))
    return output

In [24]:
model=LstmModel(len(vocab))

In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [26]:
model.to(device)

LstmModel(
  (embedding): Embedding(289, 100)
  (lstm): LSTM(100, 150, batch_first=True)
  (fc): Linear(in_features=150, out_features=289, bias=True)
)

In [27]:
epochs = 50
learning_rate = 0.001

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [32]:
# training loop

for epoch in range(epochs):
  total_loss = 0

  for batch_x, batch_y in train_dataloader:

    batch_x, batch_y = batch_x.to(device), batch_y.to(device)

    optimizer.zero_grad()

    output = model(batch_x)

    loss = criterion(output, batch_y)

    loss.backward()

    optimizer.step()

    total_loss = total_loss + loss.item()

  print(f"Epoch: {epoch + 1}, Loss: {total_loss:.4f}")

Epoch: 1, Loss: 134.3588
Epoch: 2, Loss: 120.6335
Epoch: 3, Loss: 111.5054
Epoch: 4, Loss: 103.5322
Epoch: 5, Loss: 94.8776
Epoch: 6, Loss: 86.3962
Epoch: 7, Loss: 78.0680
Epoch: 8, Loss: 69.7149
Epoch: 9, Loss: 62.1462
Epoch: 10, Loss: 55.0220
Epoch: 11, Loss: 49.1133
Epoch: 12, Loss: 43.1501
Epoch: 13, Loss: 37.7931
Epoch: 14, Loss: 33.3057
Epoch: 15, Loss: 29.2800
Epoch: 16, Loss: 25.4799
Epoch: 17, Loss: 22.4693
Epoch: 18, Loss: 19.7367
Epoch: 19, Loss: 17.3548
Epoch: 20, Loss: 15.3844
Epoch: 21, Loss: 13.8245
Epoch: 22, Loss: 12.4014
Epoch: 23, Loss: 11.1546
Epoch: 24, Loss: 10.0428
Epoch: 25, Loss: 9.2415
Epoch: 26, Loss: 8.4101
Epoch: 27, Loss: 7.7228
Epoch: 28, Loss: 7.2427
Epoch: 29, Loss: 6.6672
Epoch: 30, Loss: 6.2125
Epoch: 31, Loss: 5.8791
Epoch: 32, Loss: 5.5288
Epoch: 33, Loss: 5.2419
Epoch: 34, Loss: 4.9565
Epoch: 35, Loss: 4.8438
Epoch: 36, Loss: 4.5568
Epoch: 37, Loss: 4.3863
Epoch: 38, Loss: 4.2089
Epoch: 39, Loss: 3.9947
Epoch: 40, Loss: 3.8733
Epoch: 41, Loss: 3.75

In [33]:
model.eval()

LstmModel(
  (embedding): Embedding(289, 100)
  (lstm): LSTM(100, 150, batch_first=True)
  (fc): Linear(in_features=150, out_features=289, bias=True)
)

In [34]:
## evaluation code
total=0
correct=0
with torch.no_grad():
  for batch_features,batch_labels in test_dataloader:
    outputs=model(batch_features)
    _,predicted=torch.max(outputs.data,1)
    total=total+batch_labels.size(0)
    correct=correct+(predicted==batch_labels).sum().item()

accuracy=correct/total
print(f"accuracy: ",accuracy)

accuracy:  0.25925925925925924


In [30]:
# prediction

def prediction(model, vocab, text):

  # tokenize
  tokenized_text = word_tokenize(text.lower())

  # text -> numerical indices
  numerical_text = text_to_indices(tokenized_text, vocab)

  # padding
  padded_text = torch.tensor([0] * (61 - len(numerical_text)) + numerical_text, dtype=torch.long).unsqueeze(0)

  # send to model
  output = model(padded_text)

  # predicted index
  value, index = torch.max(output, dim=1)

  # merge with text
  return text + " " + list(vocab.keys())[index]

In [35]:
prediction(model, vocab, "The course follows a monthly")

'The course follows a monthly subscription'